**Seminar:** Struktur und Interpretation von Computerprogrammen - Grundlegende Programmierkonzepte anhand der Sprache LISP, Frühjahrsemester 2021  
**Dozenten:** Prof. Dr. Christian Tschudin, Dr. Marcel Lüthi  
**Datum:** 5. September 2021  

# Eine Krivine-Maschine in Scheme  

Luka Obser <luka.obser@unibas.ch>  
Reto Krummenacher <reto.krummenacher@unibas.ch>

### Einleitung

### 1. Theorie

### 2. Parsen und AST

### 3. Die Maschine
Die hier vorgestellte abstrakte Maschine in Scheme folgt der im original von Krivine postulierten Variante (Krivine 2007). In einem ersten Abschnitt werden die notwendigen Datenstrukturen vorgestellte, ehe auf das Verhalten der Maschiene mit ihrem Zustand und den Übergängen eingegangen wird.
#### Datenstrukturen
Die Krivine-Machine benötigt drei Typen von Datenstrukturen: Closures, einen Stack und Environments. Im Folgenden wird vertieft auf diese drei Eingegangen. 
##### Eine Closure
Ein Closure ist ein Paar bestehend aus einem als Term bezeichneten ausführbaren $\lambda$-Ausdruck und einem Environment. Die Implementierung ist mit den notwendigen *getter* und *setter* Methoden versehen, wobei *'fst* und *'snd* das erste respektive das zweite Element eines Paars returnieren.

In [2]:
; closure data structure: a pair
(define (make-closure)
  (let ((c '()))
    (lambda (msg . args)
      (cond
        ((eq? msg 'set)
          (set! c (cons (car args) (cdr args))))
        ((eq? msg 'fst)
          (car c))
        ((eq? msg 'snd)
          (cadr c))
        ((eq? msg 'get) c)
        (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))  

##### Der Stack
Auf dem Stack befinden sich Closures. Implementiert wurde dieser in Scheme als Liste. Mittels den üblichen Befehlen *'push* und *'pop* können Closures auf den Stack geschoben oder vom Stack geholt werden. Ebenfalls enthalten ist die Methode *'display*. Damit ist es möglich die einzelnen Closures auf dem Stack in der Konsole auszugeben.

In [3]:
; stack data structure: a list with push and pop
(define (make-stack)
  (let ((s '()) (n 0))
     (lambda (msg . args)  ; msg and arguments, if . used, args is seen as list which can be empty
       (cond
         ((eq? msg 'pop)
            (cond
              ((null? s)
                (display "\nERROR: Stack is empty\n"))
              (else
                (set! n (- n 1))
                (define tmpS s)
                (set! s (cdr s))
                (car tmpS)))) ;return the car element of stack
         ((eq? msg 'push)
            (set! n (+ n 1))
            (set! s (append (reverse args) s)))
         ((eq? msg 'get) s)
         ((eq? msg 'size) n)
         ((eq? msg 'display)
            (define clos (make-closure))
            (let loop ((i 0))
               (cond ((< i (length s))
                   (set! clos (list-ref s i))
                   (display "[") (display (clos 'fst)) (display ",") (display (clos 'snd)) (display "] ")
                   (loop (+ i 1))))))
         (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

##### Ein Environment
Ein Environment wurde ebenfalls als Liste implementiert. Wenn ein Environment nicht leer ist, dann ist das erste Element selbst ein Environment. Alle folgenden sind Closures, welche mittels *'append* angefügt werden können. Die Datenstruktur bietet die Möglichkeit entweder das enthaltene Environment oder die Closure an Position *k* zrückzugeben.

In [ ]:
; environment data structure: a list
(define (make-environment)
  (let ((e '()))
    (lambda (msg . args)
      (cond
        ((eq? msg 'append)
          (set! e (append e args)))  ; append takes list, thus no (car args) needed
        ((eq? msg 'getHigh)
          (car e))
        ((eq? msg 'getK)
          (list-ref e (car args))) ; get the kth closure of this environment
        ((eq? msg 'get) e)        
        (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

#### Zustand
Der Zustand der Krivine-Maschine ist das Tripel besthenden aus Term (*T*), Stack (*S*) und Environment (*E*). Der Term ist der als nächstes auszuführende $\lambda$-Ausdruck. *E* und *S* sind zu Beginn leer.
Im Unterschied zur ursprünglichen Idee von Krivine (2007, S. 4), wird in der vorgestellten Implementierung nicht mit Zeigern gearbeitet. Dies ist der Tatsache geschuldet, dass die Sprache Scheme im Gegensatz etwa zur Sprache C keine zusätzliche Zeigervariablen kennt. 

## Quellenverzeichnis
- Krivine, Jean-Louis (2007):  
  *A call-by-name lambda-calculus machine*, Springer, <https://link.springer.com/article/10.1007/s10990-007-9018-9> (letzter Zugriff 3.9.2021)
- blabla